In [1]:
import pandas as pd
import numpy as np
import time
import datetime
import sys
import requests as rq
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import json
import traceback as tb
import re

# 公式設定

In [2]:
# 計算本益比
def getPER(price, estimate_for_EPS):
    return price/estimate_for_EPS
# 計算股價淨值比
def getPBR(price, BVS):
    return price/BVS
# 計算殖利率
def getPBR(total_dividend, price):
    return total_dividend/price
# 計算股東分紅
def getTotal_dividend(cash_dividend, stock_dividend):
    return cash_dividend+stock_dividend
# 計算現距(高點)
def getDistance_from_h(highest_price, price):
    return (highest_price - price)/price
# 計算波動
def getDistance_from_h(highest_price, lowest_price):
    return (highest_price - lowest_price)/lowest_price
# 計算現距(低點)
def getDistance_from_l(Q1, Q2, Q3, Q4):
    return Q1+Q2+Q3+Q4
# 計算累積EPS
def getEPS_total(lowest_price, price):
    return (lowest_price - price)/price
# 預估EPS (還沒想好怎麼預估)
def getEstimate_for_EPS():
    pass

# 為所有欄位設定變數

In [3]:
df = pd.read_excel('StockTracker_V2.xlsx')

In [4]:
company_id_series = df['company_id']
company_name_series = df['company_name']
price_series = df['price']
PER_series = df['PER']
PBR_series = df['PBR']
BVS_series = df['BVS']
yield_rate_series = df['yield_rate']
cash_dividend_series = df['cash_dividend']
stock_dividend_series = df['stock_dividend']
total_dividend_series = df['total_dividend']
highest_price_series = df['highest_price']
distance_from_h_series = df['distance_from_h']
fluctuation_series = df['fluctuation']
distance_from_l_series = df['distance_from_l']
lowest_price_series = df['lowest_price']
last_ROE_series = df['last_ROE']
EPS_total_series = df['EPS_total']
Q1_series = df['Q1']
Q2_series = df['Q2']
Q3_series = df['Q3']
Q4_series = df['Q4']
estimate_for_EPS_series = df['estimate_for_EPS']

# 爬蟲部分

In [5]:
# 一般資料取得方式
def getData(url):
    try:
        res = rq.get(url)
        if res.status_code() == 200:
            print('good')
            return res
        else:
            print('error')
            print('[ERROR]status_code:{}'.format(res.status_code()))
            return None
    except:
        raise

# 將soup檔案轉換成json格式
def jsonLoads(soup_str):
    try:
        return json.loads(soup_str)
    except:
        raise

# 確認soup裡面是否有資料
def IsBodyEmpty(soup):
    return True if (soup.text == '\n' or len(soup.text) == 0) else False  

# 確認是否取得有公司資料的json物件
def isIDExist(jsonobj):
    return False if (jsonobj['items'][0]['vFLD_CLOSE'] == '--') else True

# 使用selenium爬取yahoo股市資料，依URL不同回傳 soup物件 或 json物件
def getDataBySelenium(url):
    try:
        driver = webdriver.PhantomJS(executable_path='C:/Users/nick800608/phantomjs-2.1.1-windows/bin/phantomjs')
        driver.get(url)
        pageSource = driver.page_source   
        soup = bs(pageSource, 'lxml')
        if soup.is_empty_element:
            print('SoupEmpty!') 
            return 0
        else:
            if IsBodyEmpty(soup):  
                print('BodyEmpty!')    
                return 0
            else:
                if soup.title.text == 'Yahoo奇摩':
                    print('url redirected!')    
                    return 0
                else:
                    return 1, soup
                
    except AttributeError:
        jsonobj = jsonLoads(soup.pre.text)
        if isIDExist(jsonobj):
            return 2, jsonobj  
        else:
            print('[Warnning]wrong id') 
            return 0 
    
    except:
        tb.print_stack()

# 設定表格取值方式

In [6]:
# 取得table左邊欄位值
def getLeftValue(subtable_list, index):
    value = subtable_list[index].select('td')[1].text
    return value

# 取得table中間欄位名(3欄)
def getMiddleColumn(subtable_list, index):
    value = subtable_list[index].select('td')[2].text
    return value

# 取得table中間欄位值(3欄)
def getMiddleValue(subtable_list, index):
    value = subtable_list[index].select('td')[3].text
    return value

# 取得table右邊欄位值
def getRightValue(subtable_list, index):
    value = subtable_list[index].select('td')[-1].text
    return value

# 設定將 "4.25元" 轉換成 float(4.25)

In [7]:
def textToFloat(pre_number):
    try:
        return float(pre_number.split('元')[0])
    except:
        return 0.0

# 設定季/EPS取得方式

In [8]:
QX_pattern = '(\d+)(?:第)(\d+)(?:季)'

def yearquarterRepl(matchobj):
    return matchobj.group(1)+matchobj.group(2)

def tupleQuarterEPS(subtable_list, index):
    pre_quarter = getMiddleColumn(subtable_list, index)
    quarter_text = re.sub(QX_pattern, yearquarterRepl, pre_quarter)
    EPS_float = textToFloat(getMiddleValue(subtable_list, index))
    return quarter_text, EPS_float

# 設定URL產生方式

In [9]:
# 產生公司基本資料URL
def getBStockURL(company_id):
    return 'https://tw.stock.yahoo.com/d/s/company_{}.html'\
    .format(company_id)  
    
# 產生個股健檢URL
def getAStockURL(company_id):
    return 'https://tw.screener.finance.yahoo.net/screener/ws?f=j&ShowID={}'\
    .format(company_id)

# 設定多進程池(非多線程) 
### 不會對字典裡同樣的K-V做修改，暫時不需要實作同步

In [10]:
def task(url):
    state_code, data = getDataBySelenium(url)
    print(state_code)
    print(data)
    # do something
    return data

In [11]:
# 單元測試
test_json_url = 'https://tw.screener.finance.yahoo.net/screener/ws?f=j&ShowID=2317'
task(test_json_url)

2
{'items': [{'vFLD_TOT_AMT1': '1681', 'BROKER_BS_BUY1': '1647', 'FLD_EPS_Q_VALUE1': '1.63', 'FLD_NET_VALUE': '64.19', 'BROKER_BS_BUY4': '484', 'FLD_EPS_Y_NAME1': '106年(近期累計)', 'vFLD_CLOSE': '104.00元', 'FLD_EPS_Y_VALUE4': '8.85', 'vFLD_YMD1': '05/22', 'vFLD_M_BAL_SHARE': '50084', 'BROKER_BS_SELL_NAME3': '元大', 'vFLD_LIST_CODE5': '2474', 'vFLD_CLOSE_MONTH': '5.67%', 'v52_WEEK_HIGH_PRICE': '105.50元', 'BROKER_BS_SELL5': '-364', 'vFLD_YMD': '05/23', 'BROKER_BS_BUY_NAME5': '德意志', 'BROKER_BS_BUY_NAME2': '摩根大通', 'vFLD_FRN_AMT3': '-8702', 'vFLD_K9_UPDNRATE': '53.94', 'vFLD_ITH_AMT3': '-392', 'vFLD_TOT_AMT3': '-9942', 'vFLD_ROE': '2.52%', 'vFLD_LIST_CODE1': '6196', 'FLD_EPS_Y_NAME2': '105年', 'BROKER_BS_BUY_NAME4': '台灣摩根', 'vFLD_FRN_AMT2': '-1844', 'vFLD_EPS': '1.63元', 'FLD_EPS_Y_VALUE3': '9.42', 'STOCK_CHARACTER': '2', 'vFLD_PBR': '1.60', 'FLD_EPS_Y_VALUE1': '1.63', 'vFLD_M_BAL_DIF': '-130', 'BROKER_BS_DATE': '05/22', 'BROKER_BS_BUY3': '712', 'BROKER_BS_SELL_NAME5': '群益', 'FLD_EPS_Q_NAME3': '105

{'count': '1',
 'error': '',
 'items': [{'BROKER_BS_BUY1': '1647',
   'BROKER_BS_BUY2': '1009',
   'BROKER_BS_BUY3': '712',
   'BROKER_BS_BUY4': '484',
   'BROKER_BS_BUY5': '322',
   'BROKER_BS_BUY_NAME1': '元富',
   'BROKER_BS_BUY_NAME2': '摩根大通',
   'BROKER_BS_BUY_NAME3': '瑞士信貸',
   'BROKER_BS_BUY_NAME4': '台灣摩根',
   'BROKER_BS_BUY_NAME5': '德意志',
   'BROKER_BS_DATE': '05/22',
   'BROKER_BS_SELL1': '-825',
   'BROKER_BS_SELL2': '-796',
   'BROKER_BS_SELL3': '-709',
   'BROKER_BS_SELL4': '-387',
   'BROKER_BS_SELL5': '-364',
   'BROKER_BS_SELL_NAME1': '日盛',
   'BROKER_BS_SELL_NAME2': '富邦',
   'BROKER_BS_SELL_NAME3': '元大',
   'BROKER_BS_SELL_NAME4': '永豐金',
   'BROKER_BS_SELL_NAME5': '群益',
   'FLD_EPS_Q_NAME1': '106年/Q1',
   'FLD_EPS_Q_NAME2': '105年/Q4',
   'FLD_EPS_Q_NAME3': '105年/Q3',
   'FLD_EPS_Q_NAME4': '105年/Q2',
   'FLD_EPS_Q_VALUE1': '1.63',
   'FLD_EPS_Q_VALUE2': '3.97',
   'FLD_EPS_Q_VALUE3': '2.00',
   'FLD_EPS_Q_VALUE4': '0.97',
   'FLD_EPS_Y_NAME1': '106年(近期累計)',
   'FLD_EPS_Y_N

In [12]:
testjsondata = {'count': '1',
 'error': '',
 'items': [{'BROKER_BS_BUY1': '4777',
   'BROKER_BS_BUY2': '3463',
   'BROKER_BS_BUY3': '879',
   'BROKER_BS_BUY4': '835',
   'BROKER_BS_BUY5': '735',
   'BROKER_BS_BUY_NAME1': '瑞士信貸',
   'BROKER_BS_BUY_NAME2': '摩根大通',
   'BROKER_BS_BUY_NAME3': '美林',
   'BROKER_BS_BUY_NAME4': '上海匯豐',
   'BROKER_BS_BUY_NAME5': '德意志',
   'BROKER_BS_DATE': '05/10',
   'BROKER_BS_SELL1': '-5704',
   'BROKER_BS_SELL2': '-2194',
   'BROKER_BS_SELL3': '-1935',
   'BROKER_BS_SELL4': '-1652',
   'BROKER_BS_SELL5': '-1272',
   'BROKER_BS_SELL_NAME1': '元富',
   'BROKER_BS_SELL_NAME2': '國泰綜合',
   'BROKER_BS_SELL_NAME3': '國票綜合',
   'BROKER_BS_SELL_NAME4': '大和國泰',
   'BROKER_BS_SELL_NAME5': '台灣匯立',
   'FLD_EPS_Q_NAME1': '105年/Q4',
   'FLD_EPS_Q_NAME2': '105年/Q3',
   'FLD_EPS_Q_NAME3': '105年/Q2',
   'FLD_EPS_Q_NAME4': '105年/Q1',
   'FLD_EPS_Q_VALUE1': '3.97',
   'FLD_EPS_Q_VALUE2': '2.00',
   'FLD_EPS_Q_VALUE3': '0.97',
   'FLD_EPS_Q_VALUE4': '1.76',
   'FLD_EPS_Y_NAME1': '105年',
   'FLD_EPS_Y_NAME2': '104年',
   'FLD_EPS_Y_NAME3': '103年',
   'FLD_EPS_Y_NAME4': '102年',
   'FLD_EPS_Y_VALUE1': '8.60',
   'FLD_EPS_Y_VALUE2': '9.42',
   'FLD_EPS_Y_VALUE3': '8.85',
   'FLD_EPS_Y_VALUE4': '8.16',
   'FLD_NET_VALUE': '62.26',
   'STOCK_CHARACTER': '2',
   'v52_WEEK_HIGH_PRICE': '105.50元',
   'v52_WEEK_LOW_PRICE': '73.00元',
   'vFLD_BAL_RATE': '3.24%',
   'vFLD_BAL_RATE_DIF': '-0.01%',
   'vFLD_BAL_YMD': '05/10',
   'vFLD_CLOSE': '104.00元',
   'vFLD_CLOSE_MONTH': '6.89%',
   'vFLD_CLOSE_SEASON': '15.68%',
   'vFLD_CLOSE_WEEK': '4.00%',
   'vFLD_D9_UPDNRATE': '79.42',
   'vFLD_DLR_AMT1': '0',
   'vFLD_DLR_AMT2': '163',
   'vFLD_DLR_AMT3': '-3458',
   'vFLD_EPS': '3.97元',
   'vFLD_FRN_AMT1': '0',
   'vFLD_FRN_AMT2': '10062',
   'vFLD_FRN_AMT3': '10945',
   'vFLD_ITH_AMT1': '0',
   'vFLD_ITH_AMT2': '61',
   'vFLD_ITH_AMT3': '1306',
   'vFLD_K9_UPDNRATE': '80.34',
   'vFLD_LIST_CODE1': '3665',
   'vFLD_LIST_CODE2': '2354',
   'vFLD_LIST_CODE3': '2474',
   'vFLD_LIST_CODE4': '6196',
   'vFLD_LIST_CODE5': '2464',
   'vFLD_M_BAL_DIF': '-32',
   'vFLD_M_BAL_SHARE': '50134',
   'vFLD_NAME_C1': '貿聯-KY',
   'vFLD_NAME_C2': '鴻準',
   'vFLD_NAME_C3': '可成',
   'vFLD_NAME_C4': '帆宣',
   'vFLD_NAME_C5': '盟立',
   'vFLD_PBR': '1.67',
   'vFLD_PER': '12.12倍',
   'vFLD_PRCQ_YMD': '2016/Q4',
   'vFLD_PROFIT': '8.44%',
   'vFLD_ROE': '6.42%',
   'vFLD_S_BAL_DIF': '-8',
   'vFLD_S_BAL_SHARE': '1626',
   'vFLD_TOT_AMT1': '0',
   'vFLD_TOT_AMT2': '10286',
   'vFLD_TOT_AMT3': '8793',
   'vFLD_TXN_VOLUME': '236266',
   'vFLD_UP_DN': '0.00',
   'vFLD_UP_DN_RATE': '0.00',
   'vFLD_YMD': '05/11',
   'vFLD_YMD1': '05/11',
   'vFLD_YMD2': '05/10',
   'vFLD_YMD3': '05/09',
   'vGET_MONEY': '3227.88億元',
   'vGET_MONEY_DATE': '2017/04',
   'vMACD': '207.66',
   'vSTK_VALUE': '62.26元'}],
 'totalcount': ''}

In [16]:
# 單元測試
fun = numbers_to_functions(1)

None


In [17]:
fun

'one'

In [14]:
def getSoupData():
    
    return "one"
 
def getJsonData():
    return "two"
 
def numbers_to_functions(state_code):
    switcher = {
        0: print('None'),        
        1: getSoupData,
        2: getJsonData,

    }
    # Get the function from switcher dictionary
    func = switcher.get(state_code, lambda: "nothing")
    # Execute the function
    return func()

In [18]:
def Multiprocessing(url_list):
    multiprocessing.freeze_support() #避免window的RuntimeError
    pool = multiprocessing.Pool()
    cpus = multiprocessing.cpu_count()
    results = []
    for i in range(0, cpus):
        result = pool.apply_async(task, args=(i,))
        results.append(result)
    pool.close()
    pool.join()
    
    return results_list

In [ ]:
rowdata_list = []

In [ ]:
def getRowdata(company_id):
    

In [ ]:


 
    

company_number_int32 = company_id_series.count()
for row_index in range(company_number_int32):
    company_id = company_id_series[row_index]    
    BStockURL = getBStockURL(company_id)
    AStockURL = getAStockURL(company_id)
    
    

In [10]:
# 測試是否有拿到Data
df.head()

,0,company_id,company_name,price,PER,PBR,BVS,yield_rate,cash_dividend,stock_dividend,...,fluctuation,distance_from_l,lowest_price,last_ROE,EPS_total,Q1,Q2,Q3,Q4,estimate_for_EPS
0,1,2317,NaN,83.5,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2324,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,2382,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,3231,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2353,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [102]:
company_number_int32 = company_id_series.count()
for row_index in range(company_number_int32):
    company_id = company_id_series[row_index]
    print(company_id)

2317
2324
2382
3231
2353
2330
3044
3189
6239
2801
2880
2881
2882
2884
2886
2887
2888
2891
2892
5880
2820
2889
2889


# 拿取公司基本資料

In [35]:
# 測試 class ComBasicData
def test_class_ComBasicData():
    testclassurl = 'https://tw.stock.yahoo.com/d/s/company_2330.html'
    driver = webdriver.PhantomJS(executable_path='phantomjs-2.1.1-windows/bin/phantomjs')
    driver.get(testclassurl)
    pageSource = driver.page_source   
    soup = bs(pageSource, 'lxml')
    
    test_basic_data = ComBasicData(soup)
        
    b_info = test_basic_data.basic_info_list
    print('[ERROR]basic_info_list') if (len(b_info) == 0) else print('[INFO]basic_info_list')
    
    b_prof = test_basic_data.profitability_list
    print('[ERROR]profitability_list') if (len(b_prof) == 0) else print('[INFO]profitability_list')
        
    b_exR_exD = test_basic_data.exR_and_exD_list
    print('[ERROR]exR_and_exD_list') if (len(b_exR_exD) == 0) else print('[INFO]exR_and_exD_list') 
    
    test_TestToFloat_str = '15.0元'
    if textToFloat(test_TestToFloat_str) == 15.0:
        print('[INFO]textToFloat')
    else:
        print('[ERROR]textToFloat')

In [36]:
test_class_ComBasicData()

[INFO]basic_info_list
[INFO]profitability_list
[INFO]exR_and_exD_list
[INFO]textToFloat


In [23]:
# This class is base-on BeautifulSoup(lxml), Selenium(webdriver PhantomJS, re)
class ComBasicData:
    def __init__(self, soup_data):
        # 取得股東權益 (yahoo>公司>基本資料)
        # 找到表格中中標題 (淺黃色的tr，3個，基本資料、獲利能力、除權息資料)
        tabletitle_list = soup_data.findAll('tr', {'bgcolor': re.compile('#FFDB6F')})
        # 基本資料
        self.basic_info_list = tabletitle_list[0].find_next_siblings()
        # 獲利能力
        self.profitability_list = tabletitle_list[1].find_next_siblings()
        # 除權息資料
        self.exR_and_exD_list = tabletitle_list[2].find_next_siblings()
    
    # 取得table左邊欄位值
    def getLeftValue(subtable_list, index):
        value = subtable_list[index].select('td')[1].text
        return value

    # 取得table中間欄位名(3欄)
    def getMiddleColumn(subtable_list, index):
        value = subtable_list[index].select('td')[2].text
        return value

    # 取得table中間欄位值(3欄)
    def getMiddleValue(subtable_list, index):
        value = subtable_list[index].select('td')[3].text
        return value

    # 取得table右邊欄位值
    def getRightValue(subtable_list, index):
        value = subtable_list[index].select('td')[-1].text
        return value
    
    #設定將 "4.25元" 轉換成 float(4.25)
    def textToFloat(pre_number_str):
        try:
            return float(pre_number_str.split('元')[0])
        except:
            return 0.0

In [ ]:
# 基本資料欄位

In [ ]:
# 現金股利

In [318]:
textToFloat(getRightValue(subtable1_list, 0))

4.0

In [319]:
# 股票股利

In [320]:
textToFloat(getRightValue(subtable1_list, 1))

1.0

In [ ]:
# 股本

In [94]:
getLeftValue(subtable1_list, 6)

'1732.87億'

In [ ]:
# 獲利能力

In [ ]:
# 取得近4季EPS list

In [131]:
Last4QEPS_list = []
for i in range(4):
    thisquarter = tupleQuarterEPS(subtable2_list, i)
    Last4QEPS_list.append(thisquarter)

In [132]:
Last4QEPS_list

[('1054', 3.98), ('1053', 2.0), ('1052', 1.02), ('1051', 1.76)]

In [ ]:
# 最近一季ROE

In [111]:
getLeftValue(subtable2_list, -1)

'6.47%'

In [ ]:
# 每股淨值

In [125]:
textToFloat(getRightValue(subtable2_list, -1).split()[-1])

62.26

# 拿取個股健檢資料

In [ ]:
# 測試URL

In [322]:
yahoosymiddataurl = 'https://tw.screener.finance.yahoo.net/screener/ws?f=j&ShowID=2317'

In [325]:
thisjsonobj = getDataBySelenium(yahoosymiddataurl)

In [ ]:
# 最後收盤價 price

In [327]:
thisjsonobj['items'][0]['vFLD_CLOSE']

'100.00元'

In [ ]:
# 近52周最高

In [329]:
thisjsonobj['items'][0]['v52_WEEK_HIGH_PRICE']

'101.50元'

In [ ]:
# 近52周最低

In [328]:
thisjsonobj['items'][0]['v52_WEEK_LOW_PRICE']

'72.50元'